# Generate Dynamic SQL Query

This notebook demonstrates how to use the Microsoft Semantic Kernel in .NET to generate dynamic SQL queries using natural language prompts and AI-powered workflows.

This notebook demonstrates how to generate and run SQL queries using the Microsoft Semantic Kernel.

Checkout the blog [Extending RAG with Dynamic SQL Queries](https://binarytrails.com/posts/2025/05/rag-with-dynamic-sql-tool) for more details:  
https://binarytrails.com/posts/2025/05/rag-with-tools


**Step 1**: Install NuGet packages

To get started with Semantic Kernel, you need to install the required NuGet packages. These packages provide the core functionality for interacting with AI models and managing environment variables. Specifically:
- `Microsoft.SemanticKernel` enables you to build and run AI-powered workflows.
- `DotNetEnv` allows you to load environment variables from a `.env` file, making it easier to manage secrets and configuration settings.

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.23.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.23.0

**Step 2**: Read environment variables

  In this step, we load these variables from a `.env` file (if present) so that they can be accessed by the application.


In [29]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\ai-examples\02-semantic-kernel-dynamic-sql-query\..\.env


**Step 3**: Instantiate the Kernel

The Semantic Kernel is the core component that orchestrates AI services and plugins. In this step, we create and configure a Kernel instance, which will be used to interact with AI models.

In [30]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = //read from environment variables
    (Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true",
    Environment.GetEnvironmentVariable("MODEL"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT"),
    Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY"),
    Environment.GetEnvironmentVariable("OPENAI_ORG_ID"));

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

**Step 4**: Create SQL Query Generation Plugin

Create a plugin  that will handle SQL generation. This plugin will contain the logic for generating SQL queries based on natural language prompts.

In [31]:
//add a semantic function using Semantic Kernel to generate a SQL query using a prompt template

var sqlPrompt = @"
You are a SQL expert. You will be given a table schema and a question. Your task is to generate a SQL query that answers the question using the provided schema.
The question will be a natural language question.
The SQL query should be in the format of a string. Do not include any additional text or explanation.
If you cannot answer the question using the provided schema, return 'SELECT NULL;'.
Schema: 
CREATE TABLE Products (
    id INT PRIMARY KEY,
    model_name VARCHAR(255),
    description VARCHAR,
    bike_type VARCHAR(50),
    price DECIMAL(10, 2),
    created_at DATETIME
);

Question: {question}
SQL Query:"; 

var sqlFunction = kernel.CreateFunctionFromPrompt(sqlPrompt,functionName: "GenerateSQLQuery");

**Step 5**: Create SQL Query Runnner Plugin

Create a plugin that will handle SQL query execution. 

In [32]:
using Microsoft.SemanticKernel;
using System.ComponentModel;
using Kernel = Microsoft.SemanticKernel.Kernel;

public class Products{
    public int Id { get; set; }
    public string ModelName { get; set; }
    public string Description { get; set; }
    public string BikeType { get; set; }
    public decimal Price { get; set; }
    public DateTime CreatedAt { get; set; }
}

public class BikeStoreDatabasePlugin
{ 
    
 [KernelFunction("execute_sql_query")]
 [Description("Executes a SQL query")]
 public List<Products> ExecuteSqlQuery(string sqlQuery)
 {
     // Here you would normally execute the SQL query against your database
     // For this example, we will just simulate the execution and return a list of products
     Console.WriteLine($"Executing SQL query: {sqlQuery}");
     
     //simulate executing a SQL query by returning a list of products
     return new List<Products>
     {
         new Products { Id = 1, ModelName = "Mountain Bike", Description = "A great mountain bike", BikeType = "Mountain", Price = 499.99m, CreatedAt = DateTime.Now },
         new Products { Id = 2, ModelName = "Road Bike", Description = "A fast road bike", BikeType = "Road", Price = 599.99m, CreatedAt = DateTime.Now }
     };     
 }

}

**Step 6**: Register the Plugin and Configure the Kernel

Register the custom plugin with the kernel so its functions are available for function calling.

In [33]:
// Register the plugin
kernel.Plugins.AddFromType<BikeStoreDatabasePlugin>();
kernel.Plugins.Add(KernelPluginFactory.CreateFromFunctions("GenerateSQLQuery", functions: [sqlFunction],description: "Plugin to generate SQL queries from natural language questions"));


**Step 7**: Ask the model about any questions about the product. You will see that a SQL query is generated and executed. In the below example, we are getting the list of all products in the database ordered by price.  

In [34]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0001
var chatCompletion = kernel.GetRequiredService<IChatCompletionService>();
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};

#pragma warning restore SKEXP0001

ChatHistory chatHistory = [];
chatHistory.AddUserMessage("Get me the list of all bikes using descending order of price");
var response = await chatCompletion.GetChatMessageContentAsync(
    chatHistory,
    executionSettings: openAIPromptExecutionSettings,
    kernel: kernel);

Console.WriteLine(response);


Executing SQL query: SELECT * FROM bikes ORDER BY price DESC;
Here is the list of all bikes in descending order of price:

1. **Road Bike**:
   - **Description**: A fast road bike
   - **Type**: Road
   - **Price**: $599.99

2. **Mountain Bike**:
   - **Description**: A great mountain bike
   - **Type**: Mountain
   - **Price**: $499.99
